1.  Import libraries and dataframes
2.  Wrangle the data
3.  Data quality and consistency checks
4.  Combine the customers data with ords_prods_merge
5.  Export as pickle file

    1.  Import libraries and dataframes

In [1]:
#Import libraries
import pandas as pd
import numpy as np
import os

In [2]:
#shortcut the main file to path
path = r'C:\Users\dodge\24-06-2023 Instacart Basket Analysis'

In [3]:
#Import customers dataframe
df = pd.read_csv(os.path.join(path, '02 Data', 'Original Data', 'customers.csv'), index_col = False)

In [4]:
#Evaluate new customers dataframe
df.shape

(206209, 10)

In [5]:
df.head()

,user_id,First Name,Surnam,Gender,STATE,Age,date_joined,n_dependants,fam_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


In [6]:
df.describe()

,user_id,Age,n_dependants,income
count,206209.000000,206209.000000,206209.000000,206209.000000
mean,103105.000000,49.501646,1.499823,94632.852548
std,59527.555167,18.480962,1.118433,42473.786988
min,1.000000,18.000000,0.000000,25903.000000
25%,51553.000000,33.000000,0.000000,59874.000000
50%,103105.000000,49.000000,1.000000,93547.000000
75%,154657.000000,66.000000,3.000000,124244.000000
max,206209.000000,81.000000,3.000000,593901.000000


In [7]:
df.columns

Index(['user_id', 'First Name', 'Surnam', 'Gender', 'STATE', 'Age',
       'date_joined', 'n_dependants', 'fam_status', 'income'],
      dtype='object')

In [8]:
df.dtypes

user_id          int64
First Name      object
Surnam          object
Gender          object
STATE           object
Age              int64
date_joined     object
n_dependants     int64
fam_status      object
income           int64
dtype: object

    2.  Wrangle the Data

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206209 entries, 0 to 206208
Data columns (total 10 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   user_id       206209 non-null  int64 
 1   First Name    194950 non-null  object
 2   Surnam        206209 non-null  object
 3   Gender        206209 non-null  object
 4   STATE         206209 non-null  object
 5   Age           206209 non-null  int64 
 6   date_joined   206209 non-null  object
 7   n_dependants  206209 non-null  int64 
 8   fam_status    206209 non-null  object
 9   income        206209 non-null  int64 
dtypes: int64(4), object(6)
memory usage: 15.7+ MB


There are multiple rows that are missing a user first name; all of the other rows have non-null counts that are consistent.

In [10]:
#Look more closely at the first name column
df['First Name'].value_counts(dropna=False)

NaN        11259
Marilyn     2213
Barbara     2154
Todd        2113
Jeremy      2104
           ...  
Merry        197
Eugene       197
Garry        191
Ned          186
David        186
Name: First Name, Length: 208, dtype: int64

Given that it's first names missing, but the rest of the fields are complete, I am going to let these move through until I get to the consistency and quality checks to see if more information can be determined there.

In [11]:
#Renaming Columns
df.rename(columns = {'First Name' : 'first_name'}, inplace = True)

In [12]:
#Rename Surname Column
df.rename(columns = {'Surnam' : "last_name"}, inplace = True)

In [13]:
#Rename State Column
df.rename(columns = {'STATE' : 'State'}, inplace = True)

In [14]:
#Rename Dependants Column
df.rename(columns = {'n_dependants' : 'dependants'}, inplace = True)

In [15]:
#Reevaluate columns
df.head()

,user_id,first_name,last_name,Gender,State,Age,date_joined,dependants,fam_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


In [16]:
#Change type of user id column to string
df['user_id'] = df['user_id'].astype('str')

In [17]:
#Review the output
df['user_id'].dtype

dtype('O')

    3. Data quality and consistency checks

In [18]:
#Review descriptive statistics again
df.describe()

,Age,dependants,income
count,206209.000000,206209.000000,206209.000000
mean,49.501646,1.499823,94632.852548
std,18.480962,1.118433,42473.786988
min,18.000000,0.000000,25903.000000
25%,33.000000,0.000000,59874.000000
50%,49.000000,1.000000,93547.000000
75%,66.000000,3.000000,124244.000000
max,81.000000,3.000000,593901.000000


There is nothing out of place in these statistics.  We will now analyze for mixed-type columns.

In [19]:
#Look for mixed type columns
for col in df.columns.tolist():
    weird = (df[[col]].applymap(type) !=df[[col]].iloc
[0].apply(type)).any(axis = 1)
    if len (df[weird]) > 0:
        print (col)

first_name


Because the "first names" column should be all names, this column should be categorized as a string.

In [20]:
#change first_name column to string instead of mixed
df['first_name'] = df['first_name'].astype('str')

We identified NaN first_names using .info and value counts earlier, but will now review the data with the is null function.

In [21]:
#Confirming missing first names
df.isnull().sum()

user_id        0
first_name     0
last_name      0
Gender         0
State          0
Age            0
date_joined    0
dependants     0
fam_status     0
income         0
dtype: int64

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206209 entries, 0 to 206208
Data columns (total 10 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   user_id      206209 non-null  object
 1   first_name   206209 non-null  object
 2   last_name    206209 non-null  object
 3   Gender       206209 non-null  object
 4   State        206209 non-null  object
 5   Age          206209 non-null  int64 
 6   date_joined  206209 non-null  object
 7   dependants   206209 non-null  int64 
 8   fam_status   206209 non-null  object
 9   income       206209 non-null  int64 
dtypes: int64(3), object(7)
memory usage: 15.7+ MB


In [23]:
#Look more closely at the first name column
df['first_name'].value_counts(dropna=False)

nan        11259
Marilyn     2213
Barbara     2154
Todd        2113
Jeremy      2104
           ...  
Merry        197
Eugene       197
Garry        191
Ned          186
David        186
Name: first_name, Length: 208, dtype: int64

This is interesting; by reclassifying the first names column from mixed type to string, it now considers "nan" a word, or pseudo-first name.  Now, let's look for any duplicates.

In [24]:
#Look for full duplicates
df_dups = df[df.duplicated()]

In [25]:
df_dups

,user_id,first_name,last_name,Gender,State,Age,date_joined,dependants,fam_status,income


There are no completely duplicated rows.

In [26]:
df.shape

(206209, 10)

    4. Combine the customers data with ords_prods_merge

I will not be able to concatenate the data as the two dataframes do not have rows/columns of the same length.  We'll need to identify a common identifier column.  

In [27]:
#Importing most up to date ords_prod_flagged dataframe to review for merging
ords_prods_flagged = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'ords_prods_flagged.pkl'))

In [28]:
#Review ords_prods_flagged for shape and matching fields/key column
ords_prods_flagged.shape

(32404859, 23)

In [29]:
ords_prods_flagged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32404859 entries, 0 to 32404858
Data columns (total 23 columns):
 #   Column                  Dtype   
---  ------                  -----   
 0   order_id                int64   
 1   user_id                 object  
 2   order_number            int64   
 3   order_day_of_week       int64   
 4   order_hour_of_day       int64   
 5   days_since_prior_order  float64 
 6   product_id              int64   
 7   add_to_cart_order       int64   
 8   reordered               int64   
 9   product_name            object  
 10  aisle_id                int64   
 11  department_id           int64   
 12  prices                  float64 
 13  _merge                  category
 14  price_range_loc         object  
 15  Busiest_day             object  
 16  Busiest_period_of_day   object  
 17  max_order               int64   
 18  loyalty_flag            object  
 19  ave_spend               float64 
 20  spending_flag           object  
 21  median

Review columns and column types for compatibility with customers.  User_id is titled the same, however it is an int64 in this dataframe.  Will change the type to string to match the customer dataframe.

In [30]:
#Change type of user id column to string
ords_prods_flagged['user_id'] = ords_prods_flagged['user_id'].astype('str')

In [31]:
#Review the output
ords_prods_flagged['user_id'].dtype

dtype('O')

In [32]:
#Review shape of ords_prods df to confirm shape difference
ords_prods_flagged.shape

(32404859, 23)

Due to RAM issues, I am going to export all of the steps above, and then reimport just the dataframes to attempt the merge.

In [33]:
#Export df (customers) file to CSV
df.to_csv(os.path.join(path, '02 Data', 'Prepared Data', 'cust_clean.csv'))

In [34]:
#Export ords_prods_flagged to pickle
ords_prods_flagged.to_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'ords_prods_flagged.pkl'))